<a href="https://colab.research.google.com/github/atsigman/data_pipeline_tutorial/blob/main/music_data_pipeline_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, clone the tutorial repo and install the music_data_pipeline package:

In [5]:
!pip install -q gdown
!gdown 1duHFr4-O12aRQZbZtBvc0NVr_tUwiNoQ -O /content/dpt_data.zip

Downloading...
From (original): https://drive.google.com/uc?id=1duHFr4-O12aRQZbZtBvc0NVr_tUwiNoQ
From (redirected): https://drive.google.com/uc?id=1duHFr4-O12aRQZbZtBvc0NVr_tUwiNoQ&confirm=t&uuid=52e12f9a-1d25-44e9-84c4-de85fe840663
To: /content/dpt_data.zip
100% 111M/111M [00:04<00:00, 26.0MB/s]


Next, download text and audio data (mounting the Google drive does not always work):

In [1]:
# First, clone the github repo for this tutorial and install music_data_pipeline:

!pip install git+https://github.com/atsigman/data_pipeline_tutorial.git@main

  Cloning https://github.com/atsigman/data_pipeline_tutorial.git (to revision main) to /tmp/pip-req-build-cuhz3o9c
  Running command git clone --filter=blob:none --quiet https://github.com/atsigman/data_pipeline_tutorial.git /tmp/pip-req-build-cuhz3o9c
  Resolved https://github.com/atsigman/data_pipeline_tutorial.git to commit 6f6693fce49be91bd8364dc76013576657eb16f0
  Preparing metadata (setup.py) ... done


Unzip the data archive and save to the appropriate subdirectory:

In [23]:
import os
import zipfile

In [14]:
ZIP_PATH = "/content/dpt_data.zip"   # location of the zip in Colab VM
EXTRACT_TO = "/content"
DATA_DIR = "/content/data"
SUBDIR_NAME = os.path.basename(ZIP_PATH)[:-4]

In [15]:
def unzip_file() -> None:
  """
  Unzips archive to target directory.
  """
  os.makedirs(DATA_DIR, exist_ok=True)

  # Unzip:
  with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
    zip_ref.extractall(EXTRACT_TO)

  # Rename subdir to "data":
  if SUBDIR_NAME != "data":
    ORIGINAL_DIR = os.path.join(EXTRACT_TO, SUBDIR_NAME)
    if os.path.exists(DATA_DIR):
      import shutil
      shutil.rmtree(DATA_DIR)  # remove if already exists
      os.rename(ORIGINAL_DIR, DATA_DIR)

  print(f"✅ Audio and text metadata extracted to: {DATA_DIR}")




In [16]:
unzip_file()

✅ Audio and text metadata extracted to: /content/data


In [17]:
# Imports:

import json
import pandas as pd
import torch
import torchaudio

from music_data_pipeline.audio_dataset import AudioDataset

Now we are ready to explore the dataset and construct a data pipeline!

# **I. Data Preprocessing Pipeline**

The goal of this module will be to analyse and preprocess the audio data and text metadata.

As we shall see, it may be necessary to a) prune the metadata in the event of invalid entries, b) add metadata, and/or c) generate new audio files.

Ultimately, the input data CSV will be converted to a dictionary, which will be saved as a JSON in the `/content/data` directory.

(In the "real world", this data would be stored to a DB, but for the sake of simplicity, we will just serialise it to a file in this tutorial.)

Let's begin by reading in and inspecting the dataset CSV:

In [25]:
df = pd.read_csv(os.path.join(DATA_DIR, "input_data.csv"))

In [26]:
df.shape

(106, 7)

In [27]:
df.columns

Index(['track_id', 'artist', 'album_title', 'genres', 'track_title', 'tempo',
       'audio_path'],
      dtype='object')

In [28]:
df.isna().sum()

,0
track_id,0
artist,0
album_title,0
genres,0
track_title,0
tempo,0
audio_path,1


In [29]:
df.head()

,track_id,artist,album_title,genres,track_title,tempo,audio_path
0,1,Alexander Sigman,VURT Cycle,"['Experimental', 'Contemporary Classical']",dlxsf,92,data/audio/000001.mp3
1,193,Ed Askew,Blue Piano,['Folk'],Here With You,49,data/audio/000193.mp3
2,207,John Cage,Cage Classics,['Experimental'],4'33,63,NaN
3,1197,Mount Eerie,Seven New Songs,['Folk'],My Burning,60,data/audio/001197.mp3
4,1683,The Sounds of Taraab,"Zanzibar, New York",['International'],Mapenzi Matamu,100,data/audio/001683.mp3


OK, so it looks as though there are 106 samples, and 7 columns (features). 1 audio path is missing.

What do you notice about the data structures for each column?

The first necessary manipulation: now that the data dir lives under `/content`, "content" should be prepended to each `audio_path`





In [33]:
df["audio_path"] = df["audio_path"].apply(lambda x: "/content/" + x if isinstance(x, str) else x)

In [34]:
df.head()

,track_id,artist,album_title,genres,track_title,tempo,audio_path
0,1,Alexander Sigman,VURT Cycle,"['Experimental', 'Contemporary Classical']",dlxsf,92,/content/data/audio/000001.mp3
1,193,Ed Askew,Blue Piano,['Folk'],Here With You,49,/content/data/audio/000193.mp3
2,207,John Cage,Cage Classics,['Experimental'],4'33,63,NaN
3,1197,Mount Eerie,Seven New Songs,['Folk'],My Burning,60,/content/data/audio/001197.mp3
4,1683,The Sounds of Taraab,"Zanzibar, New York",['International'],Mapenzi Matamu,100,/content/data/audio/001683.mp3
